## Instalando as dependências

In [10]:
%pip install pandas
%pip install boto3
%pip install python-dotenv
%pip install botocore

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Importando os modulos

In [1]:
import pandas as pd
import boto3
import os
from datetime import datetime
from dotenv import load_dotenv
from io import StringIO

## Instanciando o modulo

In [2]:
load_dotenv()

bucket = os.getenv('AWS_BUCKET_NAME')

DESTINO_S3 = 'dados/coproducoes-internacionais-participacao-brasileira.csv'

s3 = boto3.client('s3')

## Fazendo o download do arquivo no bucket e usando como dataframe

In [26]:
try:
    response = s3.get_object(Bucket=bucket, Key=DESTINO_S3)
    file_content = response['Body'].read().decode('utf-8')
    df = pd.read_csv(StringIO(file_content), sep=';')
    print("Arquivo carregado com sucesso!")
except Exception as e:
    print(f"Erro ao baixar o arquivo: {e}")



Arquivo carregado com sucesso!


## Limpando os dados

In [27]:
df = df.dropna()

df = df.drop_duplicates()

df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

for col in df.columns:
    df.loc[:1, col] = df.loc[:1, col].str.replace('"', '', regex=False)

for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].str.strip()

df = df.rename(columns={'titulo_original': 'titulo', 'data_emissao_cpb': 'data', 'acordo_producao': 'acordo producao'})

df.to_csv('assets/coproducoes-internacionais-participacao-brasileira-limpo.csv', index=False)

## Produções majoritárias emitidas após 2015

In [15]:
consulta_filtro = df[(df['part_patrimonial_brasileira'] == 'MAJORITÁRIA') & (pd.to_datetime(df['data'], format='%d/%m/%Y') > datetime(2015, 1, 1))]
print(consulta_filtro[['titulo', 'data', 'part_patrimonial_brasileira']].head())
consulta_filtro.to_csv('assets/consulta_filtro', index=False)

                     titulo        data part_patrimonial_brasileira
5                   5 CASAS  02/07/2020                 MAJORITÁRIA
11  A CIDADE ONDE ENVELHEÇO  15/03/2016                 MAJORITÁRIA
13                A ESTAÇÃO  27/06/2023                 MAJORITÁRIA
16                  A FEBRE  30/09/2019                 MAJORITÁRIA
23   A MULHER DO MEU MARIDO  28/02/2018                 MAJORITÁRIA


## Contar o número de produções por tipo de acordo de coprodução

In [29]:
consulta_agregacao = df.groupby('acordo_coproducao').size().reset_index(name='quantidade')
consulta_agregacao = consulta_agregacao.sort_values(by='quantidade', ascending=False).head(10)
print(consulta_agregacao)
consulta_agregacao.to_csv('assets/consulta_agregacao', index=False)

                                   acordo_coproducao  quantidade
8  ACORDO LATINO-AMERICANO DE COPRODUÇÃO CINEMATO...         138
7  ACORDO DE COPRODUÇÃO CINEMATOGRÁFICA ENTRE OS ...          70
5  ACORDO DE COPRODUÇÃO CINEMATOGRÁFICA ENTRE O G...          33
4  ACORDO DE COPRODUÇÃO CINEMATOGRÁFICA ENTRE O G...          18
2  ACORDO DE COPRODUÇÃO CINEMATOGRÁFICA BRASIL-AL...          17
9  AJUSTE COMPLEMENTAR ENTRE O GOVERNO DA REPÚBLI...          17
1  ACORDO DE COPRODUÇÃO AUDIOVISUAL ENTRE O GOVER...          15
6  ACORDO DE COPRODUÇÃO CINEMATOGRÁFICA ENTRE O G...          11
0  ACORDO DE COPRODUÇÃO AUDIOVISUAL ENTRE O GOVER...           1
3  ACORDO DE COPRODUÇÃO CINEMATOGRÁFICA ENTRE O B...           1


## Criar uma nova coluna categorizando os filmes como "Recente" ou "Antigo" com base no ano

In [18]:
df['categoria'] = pd.to_datetime(df['data'], format='%d/%m/%Y').apply(lambda x: 'Recente' if x.year > 2015 else 'Antigo')
consulta_condicional = df[['titulo', 'data', 'categoria']].head(10)
print(consulta_condicional)
consulta_condicional.to_csv('assets/consulta_condicional', index=False)

                     titulo        data categoria
0          A PRIMEIRA MISSA  31/07/2013    Antigo
1       EL HOMBRE QUE CUIDA  14/03/2018   Recente
2        1968 - O DESPERTAR  01/02/2019   Recente
4                31 MINUTOS  12/03/2012    Antigo
5                   5 CASAS  02/07/2020   Recente
7          A ALMA QUER VOAR  18/05/2021   Recente
8               A BARQUEIRA  28/12/2020   Recente
9            A BELA AMÉRICA  17/10/2023   Recente
10             A CASA VERDE  16/04/2009    Antigo
11  A CIDADE ONDE ENVELHEÇO  15/03/2016   Recente


## Converter a coluna 'data' para o formato datetime e criar uma nova coluna com o ano

In [19]:
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
df['ano'] = df['data'].dt.year
operacao_convercao = df[['titulo', 'data', 'ano']].head(10)
print(operacao_convercao)
operacao_convercao.to_csv('assets/operacao_convercao', index=False)

                     titulo       data   ano
0          A PRIMEIRA MISSA 2013-07-31  2013
1       EL HOMBRE QUE CUIDA 2018-03-14  2018
2        1968 - O DESPERTAR 2019-02-01  2019
4                31 MINUTOS 2012-03-12  2012
5                   5 CASAS 2020-07-02  2020
7          A ALMA QUER VOAR 2021-05-18  2021
8               A BARQUEIRA 2020-12-28  2020
9            A BELA AMÉRICA 2023-10-17  2023
10             A CASA VERDE 2009-04-16  2009
11  A CIDADE ONDE ENVELHEÇO 2016-03-15  2016


## Adicionar uma coluna com o mês de emissão em formato textual

In [20]:
df['mes_emissao'] = df['data'].dt.strftime('%B')

## Criar uma nova coluna com o título em letras maiúsculas e sem espaços extras

In [ ]:
df['titulo_formatado'] = df['titulo'].str.lower().str.strip()
operacao_string = df[['titulo', 'titulo_formatado']].head(10)
print(operacao_string)
operacao_string.to_csv('assets/operacao_string', index=False)

## Fazendo upload dos arquivos no bucket

In [22]:
def upload_to_s3(file_name, bucket, s3_key):
    try:
        s3.upload_file(file_name, bucket, s3_key)
        print(f"Arquivo {file_name} enviado para {s3_key} com sucesso!")
    except Exception as e:
        print(f"Erro ao enviar {file_name} para o bucket: {e}")

upload_to_s3('assets/coproducoes-internacionais-participacao-brasileira-limpo.csv', bucket, 'dados/coproducoes-internacionais-participacao-brasileira-limpo.csv')
upload_to_s3('assets/consulta_filtro', bucket, 'dados/consulta_filtro.csv')
upload_to_s3('assets/consulta_agregacao', bucket, 'dados/consulta_agregacao.csv')
upload_to_s3('assets/consulta_condicional', bucket, 'dados/consulta_condicional.csv')
upload_to_s3('assets/operacao_convercao', bucket, 'dados/operacao_convercao.csv')

Arquivo assets/coproducoes-internacionais-participacao-brasileira-limpo.csv enviado para dados/coproducoes-internacionais-participacao-brasileira-limpo.csv com sucesso!
Arquivo assets/consulta_filtro enviado para dados/consulta_filtro.csv com sucesso!
Arquivo assets/consulta_agregacao enviado para dados/consulta_agregacao.csv com sucesso!
Arquivo assets/consulta_condicional enviado para dados/consulta_condicional.csv com sucesso!
Arquivo assets/operacao_convercao enviado para dados/operacao_convercao.csv com sucesso!
